Toronto Neighborhood Assignment 20191216 - Coursera Data Science Capstone Course 

Part 1

In [1]:
#import libraries 
import pandas as pd   
import numpy as np


In [2]:
#read data 
toronto_data = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0] 
df_toronto = pd.DataFrame(data = toronto_data)
df_toronto.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [3]:
#Data Preparation
#filter not assigned values 
df_toronto = df_toronto[(df_toronto.Borough!= "Not assigned")]   
#Assign Borough value for the neighborhodos that == Not Assigned 
df_toronto.Neighborhood.replace("Not Assigned",df_toronto.Borough,inplace=True)

#group by Postcode and merge rows  

df_toronto_2 = df_toronto.groupby(by=['Postcode','Borough']).agg(lambda x: ','.join(x))

df_toronto_2.reset_index(level=['Postcode','Borough'], inplace=True)



In [4]:
# return head tp check neighborhoods are correctly merged
df_toronto_2.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [5]:
#return shape 
df_toronto_2.shape

(103, 3)

PART 2 - Combine LAT, LONG Data


In [6]:
#Download coordinates data
df_geo = pd.read_csv(r'C:\Users\tiina\Downloads\Geospatial_Coordinates.csv')

In [7]:
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
#rename Postal Code column on the toronto dataframe 
df_toronto_2.rename(columns={'Postcode': 'Postal Code'}, inplace=True)
    
#merge the two dataframes together based on postal code. I use inner join as the lat and long values are needed for all in the future. 

df_toronto_geo = df_toronto_2.merge(df_geo, on='Postal Code') 


In [9]:
#df data frame 
df_toronto_geo.head(12)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


In [10]:
df_toronto_geo.shape

(103, 5)

PART 3 - Analyzing Clustering the data 


In [11]:
#import new libraries  

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans 


!pip install folium
import folium
print('Libraries imported')

Libraries imported


In [12]:
# fitler data only to contain data from boroughs that contain "Toronto"

df_tor = df_toronto_geo[df_toronto_geo['Borough'].str.contains("Toronto")]
df_tor.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


Adding a hidden cell with Foursaquare credentials 

This cell defined the Foursquare credentials


In [14]:
# let's get Foursquare data for the Toronto dataset 

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = [ 
                             'Postal Code', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
#Creating a dataframe for the venues in Toronto by postal code

toronto_venues = getNearbyVenues(names=df_tor['Postal Code'],
                                   latitudes=df_tor['Latitude'],
                                   longitudes=df_tor['Longitude']                           
                                )



M4E
M4K
M4L
M4M
M4N
M4P
M4R
M4S
M4T
M4V
M4W
M4X
M4Y
M5A
M5B
M5C
M5E
M5G
M5H
M5J
M5K
M5L
M5N
M5P
M5R
M5S
M5T
M5V
M5W
M5X
M6G
M6H
M6J
M6K
M6P
M6R
M6S
M7Y
M9A


In [16]:
#examine the formed dataset 
print(toronto_venues.head())

#see how many vanues were for each Borough 
print(toronto_venues.groupby('Postal Code').count()) 

#unique categories 
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

  Postal Code   Latitude  Longitude                               Venue  \
0         M4E  43.676357 -79.293031                   Glen Manor Ravine   
1         M4E  43.676357 -79.293031  The Big Carrot Natural Food Market   
2         M4E  43.676357 -79.293031                 Grover Pub and Grub   
3         M4E  43.676357 -79.293031                       Upper Beaches   
4         M4K  43.679557 -79.352188                            Pantheon   

   Venue Latitude  Venue Longitude     Venue Category  
0       43.676821       -79.293942              Trail  
1       43.678879       -79.297734  Health Food Store  
2       43.679181       -79.297215                Pub  
3       43.680563       -79.292869       Neighborhood  
4       43.677621       -79.351434   Greek Restaurant  
             Latitude  Longitude  Venue  Venue Latitude  Venue Longitude  \
Postal Code                                                                
M4E                 4          4      4               4      

Analyzing each postal code

In [17]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Postal Code'] = toronto_venues['Postal Code'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Postal Code,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,M4E,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4K,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
#group by borough the get the frequency of categories for the borough 

toronto_grouped = toronto_onehot.groupby('Postal Code').mean().reset_index()
toronto_grouped

,Postal Code,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,M4E,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.00000,0.25000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,M4K,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.023810,0.000000,0.00,...,0.00000,0.02381,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023810
2,M4L,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,M4M,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.047619,0.000000,0.00,...,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.023810,0.000000,0.000000,0.023810
4,M4N,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,M4P,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,M4R,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619
7,M4S,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.03125,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,M4T,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,M4V,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.066667,0.000000,0.00,...,0.00000,0.00000,0.00,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000


In [19]:
# let's examine the top venues by borough 
num_top_venues = 10

for hood in toronto_grouped['Postal Code']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Postal Code'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M4E----
                        venue  freq
0           Health Food Store  0.25
1                         Pub  0.25
2                       Trail  0.25
3                Neighborhood  0.25
4                   Nightclub  0.00
5                 Men's Store  0.00
6          Mexican Restaurant  0.00
7   Middle Eastern Restaurant  0.00
8          Miscellaneous Shop  0.00
9  Modern European Restaurant  0.00


----M4K----
                    venue  freq
0        Greek Restaurant  0.21
1             Coffee Shop  0.10
2          Ice Cream Shop  0.07
3      Italian Restaurant  0.05
4  Furniture / Home Store  0.05
5               Bookstore  0.05
6              Restaurant  0.05
7             Yoga Studio  0.02
8            Dessert Shop  0.02
9         Bubble Tea Shop  0.02


----M4L----
                  venue  freq
0        Sandwich Place  0.11
1     Fish & Chips Shop  0.05
2          Liquor Store  0.05
3    Light Rail Station  0.05
4            Steakhouse  0.05
5          Burger Joint  0.05
6 

9           Italian Restaurant  0.08


----M6S----
                venue  freq
0                Café  0.09
1         Coffee Shop  0.09
2  Italian Restaurant  0.06
3    Sushi Restaurant  0.06
4           Bookstore  0.03
5        Gourmet Shop  0.03
6                Food  0.03
7          Restaurant  0.03
8           Gastropub  0.03
9  Falafel Restaurant  0.03


----M7Y----
                  venue  freq
0    Light Rail Station  0.12
1           Yoga Studio  0.06
2         Garden Center  0.06
3           Pizza Place  0.06
4            Comic Shop  0.06
5                  Park  0.06
6      Recording Studio  0.06
7            Restaurant  0.06
8        Farmers Market  0.06
9  Fast Food Restaurant  0.06




In [20]:
# create a function to order venues in desc order 
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [21]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postal Code']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
borough_venues_sorted = pd.DataFrame(columns=columns)
borough_venues_sorted['Postal Code'] = toronto_grouped['Postal Code']

for ind in np.arange(toronto_grouped.shape[0]):
    borough_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

borough_venues_sorted.head()

,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,Health Food Store,Neighborhood,Pub,Trail,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
1,M4K,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Restaurant,Bookstore,Furniture / Home Store,Indian Restaurant,Juice Bar,Liquor Store
2,M4L,Sandwich Place,Light Rail Station,Movie Theater,Burger Joint,Liquor Store,Burrito Place,Park,Fast Food Restaurant,Fish & Chips Shop,Italian Restaurant
3,M4M,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Gastropub,Yoga Studio,Food,Fish Market,Bookstore
4,M4N,Park,Lake,Swim School,Bus Line,Yoga Studio,Dog Run,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


Let's start with clustering postal codes

In [22]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Postal Code', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 4, 4, 4, 0, 4, 4, 4, 3, 4])

In [23]:
# add clustering labels   
borough_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)


toronto_merged = df_tor 

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged =toronto_merged.join(borough_venues_sorted.set_index('Postal Code'), on='Postal Code')  

toronto_merged.head()  

toronto_merged['Cluster Labels'].isna().sum() 
toronto_merged.dropna(inplace=True) 
toronto_merged

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,2.0,Health Food Store,Neighborhood,Pub,Trail,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
41,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,4.0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Restaurant,Bookstore,Furniture / Home Store,Indian Restaurant,Juice Bar,Liquor Store
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,4.0,Sandwich Place,Light Rail Station,Movie Theater,Burger Joint,Liquor Store,Burrito Place,Park,Fast Food Restaurant,Fish & Chips Shop,Italian Restaurant
43,M4M,East Toronto,Studio District,43.659526,-79.340923,4.0,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Gastropub,Yoga Studio,Food,Fish Market,Bookstore
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0.0,Park,Lake,Swim School,Bus Line,Yoga Studio,Dog Run,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197,4.0,Sandwich Place,Gym,Convenience Store,Breakfast Spot,Park,Hotel,Clothing Store,Food & Drink Shop,Eastern European Restaurant,Doner Restaurant
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,4.0,Coffee Shop,Sporting Goods Shop,Burger Joint,Salon / Barbershop,Café,Restaurant,Rental Car Location,Chinese Restaurant,Clothing Store,Yoga Studio
47,M4S,Central Toronto,Davisville,43.704324,-79.388790,4.0,Dessert Shop,Sandwich Place,Italian Restaurant,Café,Coffee Shop,Sushi Restaurant,Pizza Place,Gym,Farmers Market,Diner
48,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160,3.0,Playground,Park,Tennis Court,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
49,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049,4.0,Pub,Coffee Shop,Bagel Shop,Sports Bar,Sushi Restaurant,Restaurant,Liquor Store,Fried Chicken Joint,Supermarket,American Restaurant


In [24]:
# visualizing cluster results  
# create map
map_clusters = folium.Map(location=[43.651070, -79.347015], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Postal Code'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color= rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examine the Clusters 

In [25]:
#Cluster 1 
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Central Toronto,0.0,Park,Lake,Swim School,Bus Line,Yoga Studio,Dog Run,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


In [26]:
#Cluster 2 
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Central Toronto,1.0,Garden,Pool,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant


In [27]:
#Cluster 3 
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,2.0,Health Food Store,Neighborhood,Pub,Trail,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
64,Central Toronto,2.0,Trail,Bus Line,Park,Sushi Restaurant,Jewelry Store,Electronics Store,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Yoga Studio


In [28]:
#Cluster 4
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,Central Toronto,3.0,Playground,Park,Tennis Court,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
50,Downtown Toronto,3.0,Park,Playground,Trail,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store


In [29]:
#Cluster 5
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,East Toronto,4.0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Restaurant,Bookstore,Furniture / Home Store,Indian Restaurant,Juice Bar,Liquor Store
42,East Toronto,4.0,Sandwich Place,Light Rail Station,Movie Theater,Burger Joint,Liquor Store,Burrito Place,Park,Fast Food Restaurant,Fish & Chips Shop,Italian Restaurant
43,East Toronto,4.0,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Gastropub,Yoga Studio,Food,Fish Market,Bookstore
45,Central Toronto,4.0,Sandwich Place,Gym,Convenience Store,Breakfast Spot,Park,Hotel,Clothing Store,Food & Drink Shop,Eastern European Restaurant,Doner Restaurant
46,Central Toronto,4.0,Coffee Shop,Sporting Goods Shop,Burger Joint,Salon / Barbershop,Café,Restaurant,Rental Car Location,Chinese Restaurant,Clothing Store,Yoga Studio
47,Central Toronto,4.0,Dessert Shop,Sandwich Place,Italian Restaurant,Café,Coffee Shop,Sushi Restaurant,Pizza Place,Gym,Farmers Market,Diner
49,Central Toronto,4.0,Pub,Coffee Shop,Bagel Shop,Sports Bar,Sushi Restaurant,Restaurant,Liquor Store,Fried Chicken Joint,Supermarket,American Restaurant
51,Downtown Toronto,4.0,Coffee Shop,Café,Restaurant,Pub,Italian Restaurant,Bakery,Market,Pizza Place,Pharmacy,Pet Store
52,Downtown Toronto,4.0,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Pub,Men's Store,Mediterranean Restaurant,Hotel,Gym
53,Downtown Toronto,4.0,Coffee Shop,Park,Bakery,Pub,Café,Mexican Restaurant,Restaurant,Breakfast Spot,Shoe Store,Beer Store
